In [2]:
%matplotlib inline
import pandas as pd
import numpy as np
import os
import re


In [5]:
import joblib
import sys

from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
import numpy as np
import scipy
import seaborn as sns
import matplotlib.pylab as plt

data_directory = '../data/'
posts_path = os.path.join(data_directory,'posts_raw_cleaned','posts_raw_cleaned.csv')

df_posts = pd.read_csv(posts_path)

In [6]:
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.optimizers import RMSprop,Adam
from keras.utils import np_utils


batch_size = 32
nb_classes = 2
nb_epoch = 5


Using TensorFlow backend.


In [7]:
# LSTM for sequence classification in the IMDB dataset
import numpy
from keras.datasets import imdb
from keras.models import Sequential
from keras.layers import Dense,Input
from keras.layers import LSTM
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
# fix random seed for reproducibility


In [8]:
posts_raw_cleaned = pd.read_csv(os.path.join(data_directory,
                                             'posts_raw_cleaned', 
                                             'posts_raw_cleaned.csv'))

# Drop null

posts_raw_cleaned = posts_raw_cleaned[posts_raw_cleaned['body'].notnull()]

texts = list(posts_raw_cleaned['body'])

labels = posts_raw_cleaned['total_payout_value']

In [9]:
labels = (posts_raw_cleaned['total_payout_value'] >  np.median(posts_raw_cleaned['total_payout_value'])).astype(int)

# labels = (labels > labels.mean()).astype(int).values

In [21]:
MAX_SEQUENCE_LENGTH = 500
MAX_NB_WORDS = 20000
EMBEDDING_DIM = 100
VALIDATION_SPLIT = 0.2



In [22]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(nb_words=MAX_NB_WORDS)
tokenizer.fit_on_texts(texts)

sequences = tokenizer.texts_to_sequences(texts)

word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))


Found 413987 unique tokens.


In [23]:
from keras.utils.np_utils import to_categorical

In [24]:
data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)

labels = to_categorical(np.asarray(labels))
print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', labels.shape)

('Shape of data tensor:', (109753, 500))
('Shape of label tensor:', (109753, 2))


Link to download : http://nlp.stanford.edu/data/glove.6B.zip

In [25]:
GLOVE_DIR = os.path.join('..','word2vec_models','glove.6B')
embeddings_index = {}
f = open(os.path.join(GLOVE_DIR, 'glove.6B.100d.txt'))
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))


Found 400000 word vectors.


In [26]:
embedding_matrix = np.zeros((len(word_index) + 1, EMBEDDING_DIM))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector


In [27]:
embedding_matrix.shape

(413988, 100)

In [28]:
from keras.layers import Embedding, LSTM



In [29]:
labels = ((posts_raw_cleaned['total_payout_value'] >  np.median(posts_raw_cleaned['total_payout_value']))
          .astype(int)).values


VALIDATION_SPLIT = 0.33
# split the data into a training set and a validation set
indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]
nb_validation_samples = int(VALIDATION_SPLIT * data.shape[0])

x_train = data[:-nb_validation_samples]
y_train = labels[:-nb_validation_samples]
x_val = data[-nb_validation_samples:]
y_val = labels[-nb_validation_samples:]

y_train = to_categorical(y_train)
y_val = to_categorical(y_val)



In [30]:
from keras import backend as K
x_train = K.cast_to_floatx(x_train)
x_val = K.cast_to_floatx(x_val)

In [31]:
x_train.shape

(73535, 500)

In [35]:
import keras

In [74]:
import json
import io, json

class LossHistory(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.losses = []

    def on_batch_end(self, batch, logs={}):
        self.losses.append(logs)
#         with io.open('data.txt', 'w', encoding='utf-8') as f:
#           f.write(unicode(json.dumps(logs.__dict__, ensure_ascii=False)))
        
#         self.losses.append(logs.get('acc'))
        
#         with open("losses.txt", "w") as text_file:
#             print self.losses
#             text_file.write(self.losses)


In [75]:
x_train.shape

(73535, 500)

In [76]:
xtrain2 = x_train[0:5,:]
y_train2 = y_train[0:5,:]
xval2 = x_val[0:5,:]

y_val2 = y_val[0:5,:]

In [77]:
embedding_layer = Embedding(len(word_index) + 1,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=False)


model2 = Sequential()
model2.add(embedding_layer)
model2.add(LSTM(10, return_sequences=False,activation = 'softsign'))
model2.add(Dense(2, activation='sigmoid'))
model2.compile(loss='binary_crossentropy', optimizer='RMSprop', metrics=['accuracy'])
history = LossHistory()


history2 = model2.fit(xtrain2, y_train2, nb_epoch=2, batch_size=32,
                     validation_data=(xval2, y_val2),callbacks=[history])



Train on 5 samples, validate on 5 samples
Epoch 1/2
5/5 [==============================] - 1s - loss: 0.7310 - acc: 0.5000 - val_loss: 0.6919 - val_acc: 0.5000
Epoch 2/2
5/5 [==============================] - 0s - loss: 0.7058 - acc: 0.5000 - val_loss: 0.6874 - val_acc: 0.5000


In [80]:
for kehistory.losses[0]


{'acc': 0.5, 'batch': 0, 'loss': 0.73103034, 'size': 5}

In [ ]:
joblib.dump(history.history,os.path.join('../images','first_run' + '_100_history'))


In [ ]:
embedding_layer = Embedding(len(word_index) + 1,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=False)
for W_regularizer in [0, 0.25,0.5,0.75]:
    for U_regularizer in [0, 0.25,0.5,0.75]:
        for dropout_W in [0, 0.1,0.25]:
            for dropout_U in [0,0.1,0.25]:
                model2 = Sequential()
                model2.add(embedding_layer)
                model2.add(LSTM(100, return_sequences=False,
                                     W_regularizer = W_regularizer,
                                     U_regularizer = U_regularizer,
                                     dropout_W = dropout_W,
                                     dropout_U = dropout_U
                               ))
                model2.add(Dense(2, activation='sigmoid'))
                model2.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
                history = model2.fit(x_train, y_train, nb_epoch=50, batch_size=32,
                                     validation_data=(x_val, y_val))
                
                params ='_'.join([str(W_regularizer),
                                  str(U_regularizer),
                                  str(dropout_W),
                                  str(dropout_U)])
                
                joblib.dump(history.history,os.path.join('../images',params + '_100_history'))



In [ ]:
y_train

In [ ]:
fig,ax = plt.subplots(1,1,figsize = (8,8))
ax.plot(history.history['loss'], label = 'loss')
ax.plot(history.history['acc'], label = 'acc')
ax.plot(history.history['val_acc'], label = 'val_acc')



ax.legend()
ax.set_title('LSTM - dropout_W = 0.1, dropout_U = 0.1')
fig.savefig('results_multi_dropout_5_5.png')

## history.history